# AT&T Customer Service Demo - Home Internet Purchase

This notebook demonstrates how the **Ninai Enterprise Agentic AI Memory OS** can be used to power an AT&T customer service agent. We'll walk through a complete customer journey for purchasing Home Internet service.

## Use Case Overview

1. **Customer Identification** - Agent recognizes returning customer
2. **Context Retrieval** - Agent recalls previous interactions and preferences
3. **Service Discovery** - Agent helps find the right internet plan
4. **Order Processing** - Agent creates and tracks the order
5. **Memory Storage** - Agent stores interaction for future reference

## Setup and Configuration

In [1]:
# Install the Ninai SDK (run once)
import sys
sys.path.insert(0, "../sdk/python")

# Now import the SDK - just 2 lines!
from ninai import NinaiClient

# Configuration
NINAI_API_URL = "http://localhost:8000/api/v1"

print("✅ Ninai SDK imported successfully")

✅ Ninai SDK imported successfully


## Initialize Ninai Client

The SDK provides a clean, simple interface with just a few lines of code!

In [2]:
# ========================================
# Option 1: API Key (Production - Recommended)
# ========================================
# client = NinaiClient(api_key="nai_your_api_key_here")

# ========================================
# Option 2: Email/Password Login
# ========================================
client = NinaiClient(base_url=NINAI_API_URL)

# Login with credentials
try:
    user = client.login(email="demo@ninai.dev", password="demo1234")
    print(f"✅ Logged in as: {user.full_name} ({user.email})")
    print(f"   Organization: {user.organization_name}")
except Exception as e:
    print(f"⚠️  Login skipped (API not running): {e}")
    print("   Demo will continue with simulated responses")



✅ Logged in as: Demo User (demo@ninai.dev)
   Organization: None


## SDK Quick Reference

Here's how simple the Ninai SDK is to use:

```python
from ninai import NinaiClient

# Initialize (pick one method)
client = NinaiClient(api_key="nai_xxx")           # API Key
client = NinaiClient().login("email", "pass")     # Email/Password

# Store memories
memory = client.memories.create(
    content="Customer issue resolved",
    tags=["support", "resolved"]
)

# Search memories  
results = client.memories.search("customer issues")

# List memories
all_memories = client.memories.list(tags=["support"])

# Get/Update/Delete
memory = client.memories.get("mem_id")
client.memories.update("mem_id", tags=["updated"])
client.memories.delete("mem_id")
```

## AT&T Internet Plans

In [3]:
from dataclasses import dataclass
from typing import List

@dataclass
class Customer:
    """Customer information."""
    id: str
    name: str
    email: str
    phone: str
    address: str

@dataclass 
class InternetPlan:
    """AT&T Internet Plan."""
    name: str
    speed: str
    price: float
    features: List[str]
    plan_id: str

# Available AT&T Internet Plans
ATT_INTERNET_PLANS = [
    InternetPlan("Internet 300", "300 Mbps", 55.00, 
                 ["Unlimited data", "No contract", "Free equipment"], "ATT-INT-300"),
    InternetPlan("Internet 500", "500 Mbps", 65.00,
                 ["Unlimited data", "No contract", "Wi-Fi 6 router"], "ATT-INT-500"),
    InternetPlan("Fiber 1000", "1 Gbps", 80.00,
                 ["Unlimited data", "Symmetrical speeds", "Wi-Fi 6 router"], "ATT-FIBER-1000"),
    InternetPlan("Fiber 2000", "2 Gbps", 110.00,
                 ["Unlimited data", "Priority support", "Wi-Fi 6E router"], "ATT-FIBER-2000"),
    InternetPlan("Fiber 5000", "5 Gbps", 180.00,
                 ["Unlimited data", "Multi-gig capable", "Premium support"], "ATT-FIBER-5000"),
]

print(f"✅ Loaded {len(ATT_INTERNET_PLANS)} AT&T Internet plans")

✅ Loaded 5 AT&T Internet plans


In [4]:
from datetime import datetime
from typing import Optional, List, Dict, Any
from dataclasses import dataclass

class ATTCustomerServiceAgent:
    """AT&T Customer Service Agent powered by Ninai Memory OS SDK."""
    
    def __init__(self, ninai_client: NinaiClient):
        self.client = ninai_client  # Uses the clean SDK!
        self.current_customer: Optional["Customer"] = None
        self.conversation_history: List[Dict] = []
        
    def greet_customer(self, customer: "Customer") -> str:
        """Greet customer and check for previous interactions using Ninai SDK."""
        self.current_customer = customer
        
        # 🔥 Using Ninai SDK - just one line to search memories!
        try:
            previous_context = self.client.memories.search(
                query=f"customer {customer.email} {customer.name}",
                limit=5
            )
            has_history = len(previous_context.items) > 0
        except Exception:
            has_history = False
            previous_context = None
        
        greeting = f"Hello {customer.name}! Welcome to AT&T Customer Service.\n"
        
        if has_history and previous_context:
            greeting += "I see we've helped you before. Let me pull up your history...\n"
            for memory in previous_context.items[:2]:
                greeting += f"  • Previous: {memory.title or 'Interaction'}\n"
        else:
            greeting += "It looks like this is your first time with us. I'm here to help!\n"
        
        greeting += "\nHow can I assist you today?"
        
        self._log_conversation("agent", greeting)
        return greeting
    
    def check_service_availability(self, address: str) -> Dict[str, Any]:
        """Check what services are available at the customer's address."""
        result = {
            "address": address,
            "fiber_available": True,
            "available_plans": ATT_INTERNET_PLANS,
            "estimated_install_date": "Within 3-5 business days"
        }
        
        message = f"Great news! I checked your address at {address}.\n\n"
        if result["fiber_available"]:
            message += "🎉 **AT&T Fiber is available at your location!**\n\n"
        
        for i, plan in enumerate(result["available_plans"], 1):
            message += f"{i}. **{plan.name}** - {plan.speed} @ ${plan.price}/mo\n"
        
        self._log_conversation("agent", message)
        return result
    
    def recommend_plan(self, usage_profile: str) -> "InternetPlan":
        """Recommend an internet plan based on customer's usage."""
        usage_lower = usage_profile.lower()
        
        if any(word in usage_lower for word in ["gaming", "stream", "4k", "work from home"]):
            recommended = ATT_INTERNET_PLANS[2]  # 1 Gbps Fiber
            reason = "For gaming/streaming/WFH, I recommend our Fiber 1000 plan."
        elif any(word in usage_lower for word in ["basic", "browse", "email"]):
            recommended = ATT_INTERNET_PLANS[0]  # 300 Mbps
            reason = "For basic browsing, our Internet 300 plan is budget-friendly."
        else:
            recommended = ATT_INTERNET_PLANS[1]  # 500 Mbps
            reason = "Our Internet 500 plan is a great balanced option."
        
        message = f"\n🎯 **Recommendation: {recommended.name}**\n{reason}\n"
        message += f"Speed: {recommended.speed} | Price: ${recommended.price}/mo\n"
        
        self._log_conversation("agent", message)
        print(message)
        return recommended
    
    def create_order(self, plan: "InternetPlan", install_date: str = None) -> Dict[str, Any]:
        """Create an order and store it in Ninai Memory OS."""
        
        order = {
            "order_id": f"ORD-{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "customer": {
                "name": self.current_customer.name,
                "email": self.current_customer.email,
                "phone": self.current_customer.phone,
                "address": self.current_customer.address
            },
            "plan": {"id": plan.plan_id, "name": plan.name, "speed": plan.speed, "price": plan.price},
            "installation": {"date": install_date or "TBD", "time_window": "8 AM - 12 PM"},
            "status": "CONFIRMED",
            "created_at": datetime.now().isoformat()
        }
        
        # 🔥 Using Ninai SDK - store the order as a memory!
        try:
            memory = self.client.memories.create(
                content=f"Customer {self.current_customer.name} ordered {plan.name} at ${plan.price}/mo. Order: {order['order_id']}",
                title=f"Order: {plan.name} for {self.current_customer.name}",
                scope="personal",  # Use personal scope (no org_id needed)
                tags=["order", "internet", "home-internet", plan.plan_id],
                entities={
                    "customer_email": self.current_customer.email,
                    "order_id": order["order_id"],
                    "plan_id": plan.plan_id
                },
                metadata=order
            )
            print(f"\n💾 Memory stored: {memory.id}")
        except Exception as e:
            print(f"\n⚠️  Memory storage simulated: {e}")
        
        message = f"\n✅ **Order Confirmed!** Order ID: {order['order_id']}\n"
        message += f"Plan: {plan.name} ({plan.speed}) @ ${plan.price}/mo\n"
        message += f"Installation: {order['installation']['date']}\n"
        
        self._log_conversation("agent", message)
        print(message)
        return order
    
    def _log_conversation(self, role: str, message: str) -> None:
        self.conversation_history.append({
            "role": role, "message": message, "timestamp": datetime.now().isoformat()
        })

# Initialize the agent with the SDK client
agent = ATTCustomerServiceAgent(client)
print("✅ AT&T Customer Service Agent initialized with Ninai SDK")

✅ AT&T Customer Service Agent initialized with Ninai SDK


---

# 🎬 Demo: Customer Journey - Buying AT&T Home Internet

Let's simulate a complete customer interaction.

## Step 1: Customer Arrives

In [5]:
# Create a sample customer
customer = Customer(
    id="CUST-001",
    name="Sarah Johnson",
    email="sarah.johnson@example.com",
    phone="(555) 123-4567",
    address="123 Oak Street, Austin, TX 78701"
)

print("📞 Customer calling in...\n")
print("="*50)

# Agent greets the customer
greeting = agent.greet_customer(customer)
print(greeting)

📞 Customer calling in...

Hello Sarah Johnson! Welcome to AT&T Customer Service.
It looks like this is your first time with us. I'm here to help!

How can I assist you today?


## Step 2: Customer Inquiry

The customer expresses interest in home internet service.

In [6]:
print("👤 Customer: 'Hi! I just moved to a new house and need home internet service.")
print("             I work from home and my kids do online school, so we need something fast.'")
print()
print("="*50)
print("\n🤖 Agent checking service availability...\n")

👤 Customer: 'Hi! I just moved to a new house and need home internet service.
             I work from home and my kids do online school, so we need something fast.'


🤖 Agent checking service availability...



## Step 3: Check Service Availability

In [7]:
# Check what's available at the customer's address
availability = agent.check_service_availability(customer.address)

print("\n" + "="*50)
print(f"\n📍 Service check complete for: {customer.address}")
print(f"   Fiber Available: {'✅ Yes' if availability['fiber_available'] else '❌ No'}")
print(f"   Plans Available: {len(availability['available_plans'])}")



📍 Service check complete for: 123 Oak Street, Austin, TX 78701
   Fiber Available: ✅ Yes
   Plans Available: 5


## Step 4: Get Personalized Recommendation

In [8]:
print("👤 Customer: 'What do you recommend? We have 4 people at home,")
print("             I do video calls all day, and my kids stream and play online games.'")
print()
print("="*50)

# Get recommendation based on usage
usage_description = "work from home video calls, kids gaming and streaming, family of 4"
recommended_plan = agent.recommend_plan(usage_description)

👤 Customer: 'What do you recommend? We have 4 people at home,
             I do video calls all day, and my kids stream and play online games.'


🎯 **Recommendation: Fiber 1000**
For gaming/streaming/WFH, I recommend our Fiber 1000 plan.
Speed: 1 Gbps | Price: $80.0/mo



## Step 5: Customer Selects Plan and Creates Order

In [9]:
print("👤 Customer: 'That sounds perfect! Let's go with the Fiber 1000 plan.'")
print()
print("="*50)
print("\n🤖 Agent: 'Excellent choice! Let me set up your order...'\n")

# Create the order
order = agent.create_order(
    plan=recommended_plan,
    install_date="January 23, 2026"
)

👤 Customer: 'That sounds perfect! Let's go with the Fiber 1000 plan.'


🤖 Agent: 'Excellent choice! Let me set up your order...'


💾 Memory stored: ff8ddb32-e46b-40b2-9f90-2f26ce48a0d0

✅ **Order Confirmed!** Order ID: ORD-20260119122453
Plan: Fiber 1000 (1 Gbps) @ $80.0/mo
Installation: January 23, 2026



## Step 6: Order Summary

In [10]:
print("\n" + "="*60)
print("                    📋 ORDER SUMMARY")
print("="*60)
print()
print(f"  Order ID:        {order['order_id']}")
print(f"  Status:          {order['status']}")
print()
print("  CUSTOMER INFORMATION")
print(f"    Name:          {order['customer']['name']}")
print(f"    Email:         {order['customer']['email']}")
print(f"    Phone:         {order['customer']['phone']}")
print(f"    Address:       {order['customer']['address']}")
print()
print("  SERVICE DETAILS")
print(f"    Plan:          {order['plan']['name']}")
print(f"    Speed:         {order['plan']['speed']}")
print(f"    Monthly Cost:  ${order['plan']['price']:.2f}")
print()
print("  INSTALLATION")
print(f"    Date:          {order['installation']['date']}")
print(f"    Time Window:   {order['installation']['time_window']}")
print()
print("="*60)


                    📋 ORDER SUMMARY

  Order ID:        ORD-20260119122453
  Status:          CONFIRMED

  CUSTOMER INFORMATION
    Name:          Sarah Johnson
    Email:         sarah.johnson@example.com
    Phone:         (555) 123-4567
    Address:       123 Oak Street, Austin, TX 78701

  SERVICE DETAILS
    Plan:          Fiber 1000
    Speed:         1 Gbps
    Monthly Cost:  $80.00

  INSTALLATION
    Date:          January 23, 2026
    Time Window:   8 AM - 12 PM



## Step 7: View Conversation History

The agent maintains a complete log of the interaction for quality assurance and training.

In [11]:
print("\n📝 CONVERSATION HISTORY")
print("="*60)

for i, entry in enumerate(agent.conversation_history, 1):
    role_emoji = "🤖" if entry["role"] == "agent" else "👤"
    print(f"\n[{i}] {role_emoji} {entry['role'].upper()} @ {entry['timestamp'][:19]}")
    print("-" * 40)
    # Print first 200 chars of message
    msg = entry['message'][:200]
    if len(entry['message']) > 200:
        msg += "..."
    print(msg)


📝 CONVERSATION HISTORY

[1] 🤖 AGENT @ 2026-01-19T12:24:53
----------------------------------------
Hello Sarah Johnson! Welcome to AT&T Customer Service.
It looks like this is your first time with us. I'm here to help!

How can I assist you today?

[2] 🤖 AGENT @ 2026-01-19T12:24:53
----------------------------------------
Great news! I checked your address at 123 Oak Street, Austin, TX 78701.

🎉 **AT&T Fiber is available at your location!**

1. **Internet 300** - 300 Mbps @ $55.0/mo
2. **Internet 500** - 500 Mbps @ $65...

[3] 🤖 AGENT @ 2026-01-19T12:24:53
----------------------------------------

🎯 **Recommendation: Fiber 1000**
For gaming/streaming/WFH, I recommend our Fiber 1000 plan.
Speed: 1 Gbps | Price: $80.0/mo


[4] 🤖 AGENT @ 2026-01-19T12:24:53
----------------------------------------

✅ **Order Confirmed!** Order ID: ORD-20260119122453
Plan: Fiber 1000 (1 Gbps) @ $80.0/mo
Installation: January 23, 2026



---

# 🧠 Memory Storage Demo

This section demonstrates how the interaction is stored in Ninai Memory OS for future reference.

In [12]:
import json
# Demonstrate memory structure that would be stored
memory_example = {
    "title": f"Order Created: {recommended_plan.name} for {customer.name}",
    "content": f"""Customer {customer.name} ({customer.email}) ordered AT&T {recommended_plan.name} 
({recommended_plan.speed}) at ${recommended_plan.price}/month. Order ID: {order['order_id']}. 
Installation Address: {customer.address}. Customer usage profile: family of 4, 
work from home, kids gaming/streaming.""",
    "scope": "organization",
    "tags": ["order", "internet", "home-internet", recommended_plan.plan_id, "att-fiber"],
    "entities": {
        "customer_email": customer.email,
        "customer_name": customer.name,
        "customer_phone": customer.phone,
        "order_id": order["order_id"],
        "plan_id": recommended_plan.plan_id,
        "address": customer.address
    },
    "extra_metadata": {
        "order_details": order,
        "agent_id": "ATT-AGENT-001",
        "channel": "phone",
        "session_id": "SES-20260118-001"
    }
}

print("🧠 MEMORY OBJECT TO BE STORED")
print("="*60)
print(json.dumps(memory_example, indent=2, default=str))

🧠 MEMORY OBJECT TO BE STORED
{
  "title": "Order Created: Fiber 1000 for Sarah Johnson",
  "content": "Customer Sarah Johnson (sarah.johnson@example.com) ordered AT&T Fiber 1000 \n(1 Gbps) at $80.0/month. Order ID: ORD-20260119122453. \nInstallation Address: 123 Oak Street, Austin, TX 78701. Customer usage profile: family of 4, \nwork from home, kids gaming/streaming.",
  "scope": "organization",
  "tags": [
    "order",
    "internet",
    "home-internet",
    "ATT-FIBER-1000",
    "att-fiber"
  ],
  "entities": {
    "customer_email": "sarah.johnson@example.com",
    "customer_name": "Sarah Johnson",
    "customer_phone": "(555) 123-4567",
    "order_id": "ORD-20260119122453",
    "plan_id": "ATT-FIBER-1000",
    "address": "123 Oak Street, Austin, TX 78701"
  },
  "extra_metadata": {
    "order_details": {
      "order_id": "ORD-20260119122453",
      "customer": {
        "name": "Sarah Johnson",
        "email": "sarah.johnson@example.com",
        "phone": "(555) 123-4567",
     

## Future Interaction Benefits

When this customer calls again, the agent can instantly recall:

1. **Previous Orders** - What services they have
2. **Preferences** - Their usage patterns (gaming, streaming, work from home)
3. **History** - Past interactions and resolutions
4. **Context** - Family size, address, contact info

This enables truly personalized service! 🌟

In [13]:
# Simulate a future search
print("\n🔍 FUTURE SEARCH EXAMPLE")
print("="*60)
print("\nWhen Sarah calls again, the agent searches: 'sarah.johnson@example.com'")
print("\nReturned memories would include:")
print("  1. Order Created: Internet 1000 (Fiber) for Sarah Johnson")
print("     - Order ID: " + order['order_id'])
print("     - Tags: order, internet, home-internet, ATT-FIBER-1000")
print("     - Created: " + order['created_at'][:10])
print("\nThis context enables personalized follow-up:")
print('  🤖 "Hi Sarah! I see your fiber installation is scheduled for January 23rd."')
print('     "Is there anything I can help you with regarding your upcoming service?"')


🔍 FUTURE SEARCH EXAMPLE

When Sarah calls again, the agent searches: 'sarah.johnson@example.com'

Returned memories would include:
  1. Order Created: Internet 1000 (Fiber) for Sarah Johnson
     - Order ID: ORD-20260119122453
     - Tags: order, internet, home-internet, ATT-FIBER-1000
     - Created: 2026-01-19

This context enables personalized follow-up:
  🤖 "Hi Sarah! I see your fiber installation is scheduled for January 23rd."
     "Is there anything I can help you with regarding your upcoming service?"


## Step 8: Short-Term Memory Capture and Promotion (Customer Service)

We capture the call notes as **short-term memory (STM)** for fast retrieval. The backend TTL controls when STM is promoted into **long-term memory (LTM)**. We'll also show how to delete STM for privacy/cleanup.

In [14]:
import time
from datetime import datetime

print("🧠 Creating short-term memory for this call...")

stm_memory_id = None
try:
    stm = client.memories.create(
        title=f"Call Notes: {customer.name} - Home Internet",
        content=(
            f"Customer {customer.name} ({customer.email}) called about home internet. "
            f"Needs fast service for WFH + kids streaming/gaming. "
            f"Recommended {recommended_plan.name}. "
            f"Order {order['order_id']} scheduled for {order['installation']['date']}."
        ),
        memory_type="short_term",
        scope="personal",
        tags=["customer-service", "att", "home-internet", "call-notes", recommended_plan.plan_id],
        entities={
            "customer_email": customer.email,
            "customer_name": customer.name,
            "order_id": order["order_id"],
            "plan_id": recommended_plan.plan_id,
        },
        metadata={
            "channel": "phone",
            "agent_id": "ATT-AGENT-001",
            "created_at": datetime.now().isoformat(),
        },
        source_type="agent",
        source_id="ATT-AGENT-001",
    )
    stm_memory_id = stm.id
    print(f"✅ Short-term memory created: {stm.id}")
except Exception as e:
    print(f"⚠️  Short-term memory simulated (API not running): {e}")

print("\n📋 Listing short-term memories (STM):")
try:
    stm_list = client.memories.list(memory_type="short_term", tags=["customer-service", "att"])
    for m in stm_list.items:
        created_at = m.created_at.strftime("%Y-%m-%d %H:%M")
        print(f"- {m.title} | {m.memory_type} | {created_at}")
except Exception as e:
    print(f"⚠️  STM listing simulated: {e}")

print("\n⏳ Waiting for STM promotion (configured by server TTL)...")
time.sleep(2)

print("\n📚 Checking long-term memories (LTM) for promotion result:")
try:
    ltm_list = client.memories.list(memory_type="long_term", tags=["customer-service", "att"])
    promoted = [
        m for m in ltm_list.items
        if m.entities.get("order_id") == order["order_id"]
    ]
    if promoted:
        for m in promoted:
            print(f"✅ Promoted LTM: {m.title} | promoted={m.is_promoted}")
    else:
        print("ℹ️ No promoted LTM found yet (promotion may run asynchronously).")
except Exception as e:
    print(f"⚠️  LTM listing simulated: {e}")

print("\n🧹 Deleting the short-term call notes (privacy/cleanup):")
if stm_memory_id:
    try:
        client.memories.delete(stm_memory_id)
        print(f"✅ Deleted STM: {stm_memory_id}")
    except Exception as e:
        print(f"⚠️  Delete simulated: {e}")
else:
    print("ℹ️ No STM ID available to delete (simulated run).")

🧠 Creating short-term memory for this call...
✅ Short-term memory created: 02c922ca-0a5d-45b5-9f57-0d832e535764

📋 Listing short-term memories (STM):

⏳ Waiting for STM promotion (configured by server TTL)...

📚 Checking long-term memories (LTM) for promotion result:
ℹ️ No promoted LTM found yet (promotion may run asynchronously).

🧹 Deleting the short-term call notes (privacy/cleanup):
✅ Deleted STM: 02c922ca-0a5d-45b5-9f57-0d832e535764


---

# 📊 Summary

This demo showcased how **Ninai Enterprise Agentic AI Memory OS** powers intelligent customer service:

| Feature | Benefit |
|---------|--------|
| **Semantic Search** | Find relevant context from past interactions |
| **Entity Extraction** | Track customers, orders, products |
| **Scoped Access** | Organization-wide or personal memories |
| **Rich Metadata** | Store complete interaction details |
| **Tagging System** | Categorize and filter memories |

### Key Capabilities Demonstrated:

1. ✅ Customer identification and history retrieval
2. ✅ Contextual service recommendations  
3. ✅ Order creation and tracking
4. ✅ Memory storage for future interactions
5. ✅ Complete conversation logging

## Step 9: Long-Term Summary (Customer Service)

If the promotion job hasn't run yet, you can explicitly write a **long-term** summary to ensure the core customer context is retained.

In [15]:
summary_content = (
    f"Summary: {customer.name} ordered {recommended_plan.name} for home internet. "
    f"Order {order['order_id']} scheduled on {order['installation']['date']}. "
    "Usage profile: WFH, kids streaming/gaming. "
    f"Address: {customer.address}."
)
try:
    ltm = client.memories.create(
        title=f"LTM Summary: {customer.name} - Home Internet",
        content=summary_content,
        memory_type="long_term",
        scope="personal",
        tags=["customer-service", "att", "home-internet", "summary", recommended_plan.plan_id],
        entities={
            "customer_email": customer.email,
            "customer_name": customer.name,
            "order_id": order["order_id"],
            "plan_id": recommended_plan.plan_id,
        },
        metadata={
            "source": "customer-service-demo",
            "created_at": datetime.now().isoformat(),
        },
        source_type="agent",
        source_id="ATT-AGENT-001",
    )
    print(f"✅ Long-term summary memory created: {ltm.id}")
except Exception as e:
    print(f"⚠️  Long-term summary simulated (API not running): {e}")

✅ Long-term summary memory created: 05e60174-879a-4bdf-9791-3628220732b1


In [17]:
print("\n🔎 Searching long-term memories for this customer:")
try:
    results = client.memories.search(query=customer.email, limit=5)
    if results.items:
        for m in results.items:
            print(f"- {m.title} | type={m.memory_type} | score={m.score}")
    else:
        print("No memories found for this customer yet.")
except Exception as e:
    print(f"⚠️  Search simulated: {e}")


🔎 Searching long-term memories for this customer:
No memories found for this customer yet.


---

**Summary:**
- Short-term memories capture recent, granular customer interactions.
- Short-term memories are promoted into long-term summaries after TTL (or can be written directly).
- Short-term memories can be deleted for privacy or data minimization policies.
- This hybrid approach enables detailed recall and efficient, high-level context for future support.
